In [29]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd

In [190]:
data = pd.DataFrame(columns=['year', 'speaker', 'speech', 'n_words'])

In [191]:
data

,year,speaker,speech,n_words


In [186]:
base_url = "https://hawzah.net"
url_77 = base_url + "/fa/Subjects/47895"
url_78 = base_url + "/fa/Subjects/47894"
url_79 = base_url + "/fa/Subjects/47893"
url_80 = base_url + "/fa/Subjects/47892"

In [52]:
def get_page_links(year_url):
    request = requests.get(year_url)
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    pages_to_visit = []
    for link in soup.find_all('a'):
        link_str = link.get('href')
        if link_str and 'امامت' in link_str:
            pages_to_visit.append(base_url + link_str)
        
    return pages_to_visit

In [192]:
def get_text(pages, index, year):
    request = requests.get(pages[index])
    response_content = request.content
    soup = BeautifulSoup(response_content, 'html.parser')
    speech = []
    for p in soup.find_all('p'):
        if '<p class=' in str(p) and 'item-abstract hidden-xs' in str(p):
            break
        p = re.sub(r'<.+?>', '', str(p))
        words_lst = p.split()
        speech.extend(words_lst)
    n_words = len(speech)
    name = get_name(pages[index])
    
    return {'year': year, 'speaker': name, 'speech': speech, 'n_words': n_words}

In [175]:
def get_name(url):
    name = re.search('امامت-.+?-.+?-.+?-', url)
    if name:
        name = name.group(0).replace('-', '')
        name = name.replace('الله', '').replace('اله', '')
        name = name.replace('آیت', '').replace('امامت', '')
        
    return name

In [193]:
# year 1377
pages = get_page_links(url_77)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1377')
    data.loc[len(data.index)] = row

In [207]:
# year 1378
pages = get_page_links(url_78)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1378')
    data.loc[len(data.index)] = row

In [208]:
# year 1379
pages = get_page_links(url_79)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1379')
    data.loc[len(data.index)] = row

In [209]:
# year 1380
pages = get_page_links(url_80)
for index, _ in enumerate(pages):
    row =  get_text(pages, index, '1380')
    data.loc[len(data.index)] = row

In [210]:
data

,year,speaker,speech,n_words
0,1377,جنتی,"[-, حضور, پرشور, مردم, در, راهپیمایی, 22, بهمن...",721
1,1377,یزدی,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, الحمد, ...",6305
2,1377,رفسنجانی23,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, والصلاه...",6589
3,1377,جنتی,"[خطبه, اول., بسم, الله, الرحمن, الرحیم., والصل...",5311
4,1377,کاشانی,"[خطبه, اول, بسم, الله, الرحمن, الرحیم, نحمده, ...",5317
...,...,...,...,...
114,1380,یزدی,"[بسم, الله, الرحمن, الرحیم, الحمد, لله, رب, ال...",9231
115,1380,جنتی,"[بسم, الله, الرحمن, الرحیم, الحمد, لله, رب, ال...",4940
116,1380,هاشمی,"[بسم, الله, الرحمن, الرحیم, الحمدلله, رب, العا...",5246
117,1380,کاشانی,"[بسم, الله, الرحمن, الرحیم, الحمدالله, رب, الع...",3450


In [212]:
data.to_csv('speeches_77to80', sep='\t')